# PS1 - Population Dynamics Simulation (10 Points)

## Problem Statement
In this assignment you are going to build a simulation for population dynamics using C++ in a multithreaded environment. We have provided some starter code for you, and your job is to fill in the missing code as specified by the #TODO# blocks in the code. You can either just work in the ipynb OR you can work locally with the various files in this folder.

## Simulation Description
The world is split into `NUM_REGIONS` different regions. Each region is filled with `COMMUNITIES_PER_NUM_SPECIES` different communities of each of the `NUM_SPECIES` different species. Each of the communities is intialized with this information and a `population` and a `growth_rate` which is all packed into a `Species_Community` struct. Note that the struct also contains additional variables which you may or may not need to use depending upon your implementation.

```
typedef struct {
    std::atomic<int> population;        // the population of a speciies
    std::atomic<int> food_collected;    // the food collected in the current time period
    int region_of_world;                // region of this species community
    int species_type;                   // type of species for this species community
    float growth_rate;                  // growth_rate for this species community
    bool flag;                          // flag in case helpful to have one (you may not need this)
    std::atomic<int> atomic_helper;     // atomic in case helpful to have one (you may not need this)
    std::atomic<float> atomic_helper2;  // atomic in case helpful to have one (you may not need this)
    std::mutex mtx;                     // mutex in case helpful to have one (you may not need this)
} Species_Community;
```

For `NUM_TIMEPERIODS` the simulation runs. At each time period all of the members of each species calls the `food_oracle` inorder for everyone to `gather_all_food`. After all food is collected we can `update_all_populations` based on the amount of food gathered. In order to do so we need to first `compute_local_population_share` which is the percentage of all species WITHIN A SINGLE REGION that belong to a given species. We can then use that to `update_community_population` for each community of each species based on 3 rules as listed in later sections of this document.

When the simulation is done it prints out the populations of the various communities of species.

## Functions You'll Need To Implement
All functions you need to implement are in `util.h` and that is the only file you need to submit to gradescope!

First we'll implement helper functions:

`update_community_population` (1.5 points)

For a given community, update its population based on the input `local_population_share` and the following three rules:
1. The change in population for a community is directly proportional to its growth_rate and local_population_share
2. If it has collected enough food to feed the population it grows, else it shrinks
3. If the population drops below 5 it goes extinct

`compute_local_population_share` (1.5 points)

Returns the population share for a given community. Population share is defined as the percentage of population in a region that is a given species across all communities of all species.

Then we'll implement the overall population update step:

`update_all_populations` (3 points)
Updates the population for all communities of all species. Some quick hints/notes:
1. You will want to launch MAX_THREADS to compute this
2. You will need to use compute_local_population_share and update_community_population
3. Make sure your logic is thread safe!
4. Feel free to use helper functions if that makes your life easier!

Next we'll implement the food gathering step
`gather_all_food` (3 points)

Each simualtion step we reset the food counts to 0 and then each member of the population tries to collect food using the food_oracle(). **Here we are adding a twist of difficulty. You must use MAX_THREADS threads of parallelism to compute the updated food count for each community of each species -- not spread across the various communities.** While this isn't the naturally most efficient way to code it up, it will make you have to use some amount of synchronization!

Finally we'll implement the main function
`population_dynamics` (1 point)

Here you'll just need to make sure to gather food and update the population for all NUM_TIME_PERIODS

## Submission
Once you are done, download and submit (or just submit if you are working locally) your `util.h` file to **Courseworks**! As we do not have an autograder we can't use Gradescope.

## Notes and Hints
+ **DO NOT CHANGE FUNCTION DEFINITIONS** or you will break our grading scripts
+ See the syllabus for our course collaboration policy (long story short you are welcome to collaborate at a high level but please do not copy each others code).
+ If you are working in Colab, you can change the formatting of the code to different color schemes: just change the `%%cpp -n <filename>.h -s xcode` to a different `-s` flag. The list can be [found at this link](https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b).
+ If you are running the code locally, note that you can compile and execute with the run.sh script. If you get errors of the form `error: no matching constructor for initialization of 'std::thread'` when you attempt to do this (likely to happen if you are on Mac), you can add the extra compile flag `-std=c++11` and it should resolve the issue.
+ Please reach out on Slack with any and all questions!

In [ ]:
# Install some magic to make c programs look nice!
!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py

In [ ]:
%load_ext cpp_plugin

In [ ]:
#@title Helper functions an constants
%%cpp -n helpers.h -s xcode

#include <iostream>
#include <thread>
#include <vector>
#include <mutex>
#include <atomic>
#include <cstdlib>
#include <cmath>

// Some helpful constants
#define NUM_REGIONS 2
#define NUM_SPECIES 2
#define COMMUNITIES_PER_NUM_SPECIES 2
#define COMMUNITIES_PER_REGION (NUM_SPECIES*COMMUNITIES_PER_NUM_SPECIES)
#define TOTAL_COMMUNITIES (NUM_REGIONS*COMMUNITIES_PER_REGION)
#define MAX_STARTING_POPULATION 10
#define NUM_TIME_PERIODS 5

/************* 
 * With Presets of 2,2,2,10,5 you should get
 * 
 * ID[0]: of type [1]: in region [0]: had final population [11]
 * ID[1]: of type [1]: in region [0]: had final population [14]
 * ID[2]: of type [1]: in region [1]: had final population [237]
 * ID[3]: of type [0]: in region [1]: had final population [9]
 * ID[4]: of type [0]: in region [0]: had final population [97]
 * ID[5]: of type [0]: in region [0]: had final population [24]
 * ID[6]: of type [0]: in region [0]: had final population [5]
 * ID[7]: of type [1]: in region [1]: had final population [218]
 *
 * OR, running on Mac you may get:
 * ID[0]: of type [1]: in region [0]: had final population [14]
 * ID[1]: of type [0]: in region [0]: had final population [81]
 * ID[2]: of type [0]: in region [0]: had final population [43]
 * ID[3]: of type [0]: in region [0]: had final population [23]
 * ID[4]: of type [0]: in region [1]: had final population [14]
 * ID[5]: of type [0]: in region [1]: had final population [170]
 * ID[6]: of type [1]: in region [1]: had final population [8]
 * ID[7]: of type [0]: in region [0]: had final population [5]
 * ************/

// data structure to store information about each species
typedef struct {
    std::atomic<int> population;        // the population of a speciies
    std::atomic<int> food_collected;    // the food collected in the current time period
    int region_of_world;                // region of this species community
    int species_type;                   // type of species for this species community
    float growth_rate;                  // growth_rate for this species community
    bool flag;                          // flag in case helpful to have one (you may not need this)
    std::atomic<int> atomic_helper;     // atomic in case helpful to have one (you may not need this)
    std::atomic<float> atomic_helper2;  // atomic in case helpful to have one (you may not need this)
    std::mutex mtx;                     // mutex in case helpful to have one (you may not need this)
} Species_Community;

// food oracle function call
// call this with a community id to get a "random" amount of food back
// this represents one community member going out to get food
// we hardcode to 1 for determinism in testing but in theory should be random
int food_oracle(int community_id){return 1;};

// random range integer in range [0,max_range)
int randRange(int max_range){return rand() % max_range;}
// random float in range [0,1]
float rand01(){return (float)rand() / (float)RAND_MAX;}

In [ ]:
#@title Utility functions to be implemented
%%cpp -n util.h -s xcode

#include "helpers.h"
// I suggest testing with this set to 1 first!
#define MAX_THREADS 8

// function to simulate population change for one community of one species
//
// Note: 1) The change in population for a community is proportional to 
//          its growth_rate and local_population_share
//       2) If it has collected enough food to feed the population it grows, else it shrinks
//       3) If the population drops below 5 it goes extinct
void update_community_population(Species_Community *communities, int community_id, float local_population_share) {
  //
  // # TODO: add implementation
  //
}

// function to find the local population share for one community of one species
//
// Note: 1) Population share is defined as the percentage of population in a region
//          that is a given species across all communities of all species
float compute_local_population_share(Species_Community *communities, int community_id){
  //
  // # TODO: add implementation
  //
}

// Updates the population for all communities of all species
//
// Note: 1) You will want to launch MAX_THREADS to compute this
//       2) You will need to use compute_local_population_share and update_community_population
//       3) Make sure your logic is thread safe! Warning there likely is a data dependancy!
//       4) Feel free to use helper functions if that makes your life easier!
void update_all_populations(Species_Community *communities){
  //
  // # TODO: add implementation
  //
}

// function to simulate food gathering
// 
// Note: 1) Each round food starts at 0 and each member of the population tries to collect food
//       2) Please use food_oracle() to get a new amount of food for each member of the population
//       3) Please use MAX_THREADS threads per Species_Community! (Not spread across them but for each one)
//       4) All other implementation details are up to you!
void gather_all_food(Species_Community *communities) {
  //
  // # TODO: add implementation
  //
}

// the main function
//
// Hints: gathers food and updates the population for everyone
//        for NUM_TIME_PERIODS
void population_dynamics(Species_Community *communities){
  //
  // # TODO: add implementation
  //
}

In [ ]:
#@title The main runner which executes your code.
%%cpp -n run.cpp -s xcode

#include "util.h"

int main() {
  // initialize random
  srand(1337);
  Species_Community communities[TOTAL_COMMUNITIES];
  
  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    communities[community_id].population = randRange(MAX_STARTING_POPULATION) + 5;
    communities[community_id].region_of_world = randRange(NUM_REGIONS);
    communities[community_id].species_type = randRange(NUM_SPECIES);
    communities[community_id].growth_rate = rand01();
  }

  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    std::cout << "ID[" << community_id << "]: of type [" << communities[community_id].species_type <<
                 "]: in region [" << communities[community_id].region_of_world << "]: had initial population [" << 
                 communities[community_id].population << "]" << std::endl;
  }
  
  // run the simulation
  population_dynamics(communities);
  
  // print the final populations
  std::cout << "\n---------\n---------\n";
  for (int community_id = 0; community_id < TOTAL_COMMUNITIES; community_id++){
    std::cout << "ID[" << community_id << "]: of type [" << communities[community_id].species_type <<
                 "]: in region [" << communities[community_id].region_of_world << "]: had final population [" << 
                 communities[community_id].population << "]" << std::endl;
  }
  return 0;
}

In [ ]:
!g++ run.cpp -o run.exe -lpthread
!./run.exe